### Problem statement

An area of an interconnected power system has three units operating on economic dispatch for a total demand of 850 MW. The variable operating heat rates and fuel costs are given by :

$H_{1}$ = 510 + 7.2 * $P_{1}$ + (0.00142 * $P_{1}^{2}$) MBtu / hr , Fuel Cost (F) = 1.1 Rs / MBtu , 150 <= $P_{1}$ <= 600 MW

$H_{2}$ = 310 + 7.85 * $P_{2}$ + (0.00194 * $P_{2}^{2}$) MBtu / hr , Fuel Cost = 1 Rs / MBtu , 100 <= $P_{2}$ <= 400 MW

$H_{3}$ = 78 + 7.97 * $P_{3}$ + (0.00482 * $P_{3}^{2}$) MBtu / hr , Fuel Cost = 1 Rs / MBtu , 50 <= $P_{3}$ <= 200 MW

 
 


where $P_{1}$,  $P_{2}$ and  $P_{3}$  are in MW. Determin the power output of each unit, the incremental operating costs, and the total operating cost $C_{T}$ that minimizes $C_{T}$ for a total demand of 850 MW. Transmission losses are neglected.

## Calculating economic dispatch solution neglecting line losses

This problem is solved using the SymPy module in Python. 

Reference link on creating equations and solving them: 

https://stackoverflow.com/questions/30791504/python-partial-derivatives-easy 

https://problemsolvingwithpython.com/10-Symbolic-Math/10.07-Solving-Two-Equations-for-Two-Unknowns/

https://docs.sympy.org/latest/modules/solvers/solvers.html

https://docs.sympy.org/latest/guides/solving/solve-equation-algebraically.html

https://docs.sympy.org/latest/modules/solvers/solvers.html#sympy.solvers.solvers.solve






In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sympy import symbols, diff, Eq, solve, Symbol


Operating costs of generators can be derived from the heat rate and fuel costs as $C_{i}$ = H * F

Thus :
$C_{i}$ = $H_{i} * F$
    

In [ ]:
# Total demand
PTotal = 850 # in MW

In [6]:
# Adding coefficients for cost functions:

# heat function format = a * P + b * P^2 + c
# for unit 1
aP1 = 7.2
bP1 = 0.00142
cP1 = 510
f1 = 1.1 # fuel cost for unit 1
# power limits in MW
P1_LL = 150 # lower limit
P1_UL = 600 # upper limit

# for unit 2
aP2 = 7.85
bP2 = 0.00194
cP2 = 310
f2 = 1 # fuel cost for unit 2
# power limits in MW
P2_LL = 100 # lower limit
P2_UL = 400 # upper limit

# for unit 3
aP3 = 7.97
bP3 = 0.00482
cP3 = 78
f3= 1 # fuel cost for unit 3
# power limits in MW
P3_LL = 50 # lower limit
P3_UL = 200 # upper limit



In [4]:
# define a function to model the cost function of the generators. 
# a and b are coefficients of the cost function input P ( generator input power)
def costFunction(pT, a, b):
    cost = (a*pT) + (b*(pT**2))
    return cost

## Calculating individual functions using Numpy

In [22]:
# defining the transmission loss function of the power system

p1, p2, p3 = symbols('p1 p2 p3', real = True) # symbols for the power outputs
lam = Symbol('lam', real = True) # symbol for the lambda = dC / dP



In [16]:
def generateCostFunction(a, b, c, f, symbol):
    genCostFunc = ((a * symbol) + (b * symbol**2) + c) * f
    return genCostFunc

c1 = generateCostFunction(aP1, bP1, cP1, f1, p1) # in Rs / hr
c2 = generateCostFunction(aP2, bP2, cP2, f2, p2) # in Rs / hr
c3 = generateCostFunction(aP3, bP3, cP3, f3, p3) # in Rs / hr

In [17]:
costs = [c1, c2, c3]

for c in costs:
    print(c)

0.001562*p1**2 + 7.92*p1 + 561.0
0.00194*p2**2 + 7.85*p2 + 310
0.00482*p3**2 + 7.97*p3 + 78


We know that : 
$$\lambda = \frac{dC_{1}}{ dP_{1}} = \frac{dC_{2}}{ dP_{2}} = \frac{dC_{3}}{ dP_{3}}$$


In [23]:
# writing a function that does differentiation of an equation
# inputs to the function are the equation to be differentiated and the variable (sym) wrt which the equation 
# is differentiated
def calculateDiff(eqn, sym):
    dEq_dSym = diff(eqn, sym)
    return dEq_dSym



In [24]:
dC_dP1 = Eq(calculateDiff(c1, p1), lam) # creating the eqquation for Unit 1
dC_dP2 = Eq(calculateDiff(c2, p2), lam) # Creating the equation for unit 2
dC_dP3 = Eq(calculateDiff(c3, p3), lam) # creating the equation for unit 3
pTotal_eq = Eq(p1 + p2 + p3, 850) # equatin for total demand

print(dC_dP1)
print(dC_dP2)

Eq(0.003124*p1 + 7.92, lam)
Eq(0.00388*p2 + 7.85, lam)


In [25]:
solutionsDict = solve((dC_dP1, dC_dP2, dC_dP3, pTotal_eq),(p1, p2, p3, lam))
solutionsDict

{p1: 393.169836945603,
 p2: 334.603755313934,
 p3: 122.226407740463,
 lam: 9.14826257061806}

In [32]:
# Check if the values are within the generator limits
def checkLimits(value, lowerLimit, upperLimit):
    if value >= lowerLimit and value <= upperLimit:
        return True
    else:
        return False
    
limitDict = {solutionsDict[p1]: [P1_LL, P1_UL], 
             solutionsDict[p2]: [P2_LL, P2_UL], 
             solutionsDict[p3]: [P3_LL, P3_UL]}
inLimit = []

for key, value in limitDict.items():
    inLimit.append(checkLimits(key, value[0], value[1]))

print(inLimit)
unitsList = ['P1', 'P2', 'P3']
for index,unit in enumerate(unitsList):
    if inLimit[index] == True:
        print(f'{unit} is within operating limits')
    else:
        print(f'{unit} is out of limits')
        
    


[True, True, True]
P1 is within operating limits
P2 is within operating limits
P3 is within operating limits
